In [14]:
#r "nuget:FParsec"
open FParsec

Installed Packages FParsec, 1.1.1

In [15]:
let input = File.ReadAllText "inputs/11/input.txt"

type Op = Old | Const of int
type Monkey = 
  {
    Items: int list
    Operation: char * Op
    Test: int * int * int
    BusinessDone: int;
    WorryDecay: int64 -> int
  }
  with 
    member m.doBusiness x = 
      let (op, operand) = m.Operation
      (match op with '*' -> (*) | '+' -> (+)) 
        (int64 x)
        (int64 (match operand with Old -> x | Const n -> n)) |> m.WorryDecay
    member m.pickTarget x =
      let (divisor, ifTrue, ifFalse) = m.Test
      if x % divisor = 0 then ifTrue else ifFalse
    static member catch item (m:Monkey) = { m with Items = item :: m.Items}

let monkeyId = skipCharsTillStringCI "monkey " true 50 >>. pint32
let startingItems = skipCharsTillString ": " true 50 >>. sepBy (pint32) (pstring ", ")
let operation =  
  (skipCharsTillString "= old " true 50 >>. anyChar .>> spaces) .>>. ((pstring "old" >>% Old) <|> (pint32 |>> Const))
let test = tuple3 (skipCharsTillString "divisible by " true 50 >>. pint32) monkeyId monkeyId

let monkeyParser = 
  monkeyId >>. pipe3 startingItems operation test (fun i o t -> 
  {
    Items = i;
    Operation = o; 
    Test = t; 
    BusinessDone = 0;
    WorryDecay = fun x -> int (x / 3L)
  })

let monkeys = 
  match run (many monkeyParser) input with
  | Success(monkeys,_,_) -> monkeys
  | Failure(e,_,_) -> failwith e
  |> Seq.toArray

let throw target item monkeys = monkeys |> Array.updateAt target (Monkey.catch item monkeys[target]) 

let takeTurn (monkeys: Monkey array) id =
  let monkey = monkeys[id]
  monkey.Items
  |> Seq.map (monkey.doBusiness)
  |> Seq.map (fun i -> monkey.pickTarget i,i)
  |> Seq.fold (fun m (t,i) -> throw t i m) monkeys
  |> Array.updateAt id {monkey with Items = []; BusinessDone = monkey.BusinessDone + monkey.Items.Length}

let doRound monkeys =
  Seq.fold takeTurn monkeys [0..monkeys.Length-1]

let monkeyBusinessLevel = 
  Seq.map (fun x -> int64 x.BusinessDone)
  >> Seq.sortDescending
  >> Seq.take 2
  >> Seq.reduce (*)

let repeat n f = Seq.foldBack (fun () -> f) (Seq.replicate n ()) 

monkeys
|> repeat 20 doRound
|> monkeyBusinessLevel


61503

In [16]:
let bigModulo = 
  monkeys 
  |> Seq.map (fun {Test = (d,_,_)} -> int64 d) 
  |> Seq.distinct
  |> Seq.reduce (*)

let moreMonkeys = 
  monkeys
  |> Array.map (fun m -> { m with WorryDecay = (fun x -> int (x % bigModulo))})

moreMonkeys
|> repeat 10000 doRound
|> monkeyBusinessLevel


14081365540